In [1]:
import sys
sys.path.append('../MaskRCNN/')
import itertools
import numpy as np
import shutil
import cv2
import six
assert six.PY3, "FasterRCNN requires Python 3!"
import tensorflow.compat.v1 as tf
import tqdm
import time
import subprocess
import os

import tensorpack_viz as tpviz
from tensorpack_tfutils import get_tf_version_tuple, get_model_loader
from tensorpack_utils import fix_rng_seed
from tensorpack_input_source import QueueInput
from tensorpack_train import TrainConfig
from tensorpack_interface import launch_train_with_config
from tensorpack_callbacks import PeriodicCallback, EnableCallbackIf, ModelSaver,\
                                 GraphProfiler, PeakMemoryTracker, EstimatedTimeLeft, SessionRunTimeout, \
                                 MovingAverageSummary, ProgressBar, MergeAllSummaries, RunUpdateOps, ScheduledHyperParamSetter
import tensorpack_logger as logger


from dataset import DetectionDataset
from config import finalize_configs, config as cfg
from data import get_eval_dataflow, get_train_dataflow, get_batch_train_dataflow
from eval import DetectionResult, predict_image, multithread_predict_dataflow, EvalCallback
from viz import draw_annotation, draw_final_outputs, draw_predictions, draw_proposal_recall
from performance import ThroughputTracker, humanize_float
from model.generalized_rcnn import ResNetFPNModel
import horovod.tensorflow as hvd

config = ['MODE_MASK=True',
'MODE_FPN=True',
'DATA.BASEDIR=/workspace/shared_workspace/data/coco/coco/',
'DATA.TRAIN=["train2017"]',
'DATA.VAL=("val2017",)',
'TRAIN.BATCH_SIZE_PER_GPU=4',
'TRAIN.LR_EPOCH_SCHEDULE=[(8, 0.1), (10, 0.01), (12, None)]',
'TRAIN.EVAL_PERIOD=24',
'TRAIN.BACKBONE_NCHW=False',
'TRAIN.FPN_NCHW=False',
'TRAIN.RPN_NCHW=False',
'TRAIN.MASK_NCHW=False',
'RPN.TOPK_PER_IMAGE=True',
'PREPROC.PREDEFINED_PADDING=False',
'BACKBONE.WEIGHTS=/workspace/shared_workspace/data/coco/pretrained-models/ImageNet-R50-AlignPadding.npz',
'BACKBONE.NORM=FreezeBN',
'TRAIN.WARMUP_INIT_LR=0.000416666666667',
'FRCNN.BBOX_REG_WEIGHTS=[20., 20., 10., 10.]',
'TRAINER=horovod']

In [2]:
cfg.update_args(config)

In [3]:
MODEL = ResNetFPNModel(True)

In [6]:
is_horovod = cfg.TRAINER == 'horovod'

In [7]:
hvd.init()

In [8]:
finalize_configs(is_training=True)

[0819 10:39:50 @config.py:264] WRN It's not recommended to use horovod for single-machine training. Replicated trainer is more stable and has the same efficiency.
[0819 10:39:50 @config.py:285] Config: ------------------------------------------
{'BACKBONE': {'FREEZE_AFFINE': False,
              'FREEZE_AT': 2,
              'NORM': 'FreezeBN',
              'RESNET_NUM_BLOCKS': [3, 4, 6, 3],
              'STRIDE_1X1': False,
              'TF_PAD_MODE': False,
              'WEIGHTS': '/workspace/shared_workspace/data/coco/pretrained-models/ImageNet-R50-AlignPadding.npz'},
 'DATA': {'BASEDIR': '/workspace/shared_workspace/data/coco/coco/',
          'CLASS_NAMES': ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
                          'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
                          'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
                          'elephant', 'bear', 'zebra', 'gira

In [9]:
cfg.TRAIN.EVAL_PERIOD = 1
tf.set_random_seed(cfg.TRAIN.SEED)
fix_rng_seed(cfg.TRAIN.SEED*hvd.rank())
np.random.seed(cfg.TRAIN.SEED)

In [10]:
images_per_epoch = 120000
images_per_step = cfg.TRAIN.NUM_GPUS * cfg.TRAIN.BATCH_SIZE_PER_GPU
steps_per_epoch = images_per_epoch // images_per_step
batch_size_lr_factor = images_per_step # The LR is defined for bs=1 and then scaled linearly with the batch size
base_lr_adjusted_for_bs = cfg.TRAIN.BASE_LR * batch_size_lr_factor

In [11]:
# Warmup LR schedule is step based
warmup_start_step = 0
warmup_end_step = cfg.TRAIN.WARMUP_STEPS
warmup_start_lr = cfg.TRAIN.WARMUP_INIT_LR*8
warmup_end_lr = base_lr_adjusted_for_bs
warmup_schedule = [(warmup_start_step, warmup_start_lr), (warmup_end_step, warmup_end_lr)]

In [12]:
warmup_end_epoch = cfg.TRAIN.WARMUP_STEPS * 1. / steps_per_epoch
training_start_epoch = int(warmup_end_epoch + 0.5)
lr_schedule = [(training_start_epoch, base_lr_adjusted_for_bs)]

In [13]:
max_epoch = None
for epoch, scheduled_lr_multiplier in cfg.TRAIN.LR_EPOCH_SCHEDULE:
    if scheduled_lr_multiplier is None:
        max_epoch = epoch # Training end is indicated by a lr_multiplier of None
        break

    absolute_lr = base_lr_adjusted_for_bs * scheduled_lr_multiplier
    lr_schedule.append((epoch, absolute_lr))

In [14]:
train_dataflow = get_batch_train_dataflow(cfg.TRAIN.BATCH_SIZE_PER_GPU)

In train dataflow
loading annotations into memory...
Done (t=18.26s)
creating index...
index created!
[0819 10:40:57 @dataset.py:50] Instances loaded from /workspace/shared_workspace/data/coco/coco/annotations/instances_train2017.json.


100%|██████████| 118287/118287 [00:19<00:00, 6175.54it/s]

[0819 10:41:17 @tensorpack_utils.py:349] Load Load annotations for train2017 finished, time:19.2508sec.


Done loading roidbs
[0819 10:41:20 @data.py:618] Filtered 1021 images which contain no non-crowd groudtruth boxes. Total #images for training: 117266
Batching roidbs
Done batching roidbs


In [16]:
train_dataflow.reset_state()

In [17]:
img = next(iter(train_dataflow))

In [19]:
img.keys()

dict_keys(['anchor_labels_lvl2', 'anchor_boxes_lvl2', 'anchor_labels_lvl3', 'anchor_boxes_lvl3', 'anchor_labels_lvl4', 'anchor_boxes_lvl4', 'anchor_labels_lvl5', 'anchor_boxes_lvl5', 'anchor_labels_lvl6', 'anchor_boxes_lvl6', 'images', 'orig_image_dims', 'orig_gt_counts', 'gt_labels', 'gt_boxes', 'filenames', 'gt_masks'])

In [21]:
img['images'].shape

(4, 800, 1099, 3)

In [24]:
from model.backbone import resnet_fpn_backbone
from utils.randomnness import SeedGenerator
seed_gen = SeedGenerator(cfg.TRAIN.SEED)

In [25]:
resnet_fpn_backbone(img['images'], cfg.BACKBONE.RESNET_NUM_BLOCKS, seed_gen=seed_gen)

[0819 10:45:16 @tensorpack_models.py:181] conv0 input: [4, 805, 1125, 3]
Use channels_last data format
Instructions for updating:
Please use `layer.__call__` method instead.


AttributeError: 'NoneType' object has no attribute 'is_training'